<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [ ]:
# pip install twilio


In [1]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [2]:
query = 'Argentina'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=f3cf65ff7ae64bccad9210657220807&q=Argentina&days=1&aqi=no&alerts=no'

In [3]:
response = requests.get(url_clima).json()

In [4]:
response

{'location': {'name': 'Buenos Aires',
  'region': 'Distrito Federal',
  'country': 'Argentina',
  'lat': -34.59,
  'lon': -58.67,
  'tz_id': 'America/Argentina/Buenos_Aires',
  'localtime_epoch': 1674945051,
  'localtime': '2023-01-28 19:30'},
 'current': {'last_updated_epoch': 1674944100,
  'last_updated': '2023-01-28 19:15',
  'temp_c': 28.0,
  'temp_f': 82.4,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 13.6,
  'wind_kph': 22.0,
  'wind_degree': 90,
  'wind_dir': 'E',
  'pressure_mb': 1013.0,
  'pressure_in': 29.91,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 70,
  'cloud': 0,
  'feelslike_c': 29.7,
  'feelslike_f': 85.4,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 7.0,
  'gust_mph': 10.3,
  'gust_kph': 16.6},
 'forecast': {'forecastday': [{'date': '2023-01-28',
    'date_epoch': 1674864000,
    'day': {'maxtemp_c': 31.2,
     'maxtemp_f': 88.2,
     'mintemp_c': 21.6,
     'm

In [46]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [47]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [ ]:
len(response['forecast']['forecastday'][0]['hour'])

In [ ]:
response['forecast']['forecastday'][0]['hour'][1]

In [ ]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

In [ ]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

# Dataframe

In [5]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [6]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<00:00, 48003.48it/s]


In [50]:
datos[0]

('2023-01-28', 0, 'Patchy rain possible', 22.0, 0, 63)

In [7]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-01-28,0,Patchy rain possible,22.0,0,63
1,2023-01-28,1,Partly cloudy,22.3,0,0
2,2023-01-28,2,Cloudy,22.2,0,0
3,2023-01-28,3,Light rain shower,22.0,0,51
4,2023-01-28,4,Partly cloudy,21.9,0,0
5,2023-01-28,5,Clear,21.8,0,0
6,2023-01-28,6,Partly cloudy,21.6,0,0
7,2023-01-28,7,Partly cloudy,22.2,0,0
8,2023-01-28,8,Partly cloudy,23.5,0,0
9,2023-01-28,9,Partly cloudy,25.1,0,0


In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [8]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
12,2023-01-28,12,Patchy rain possible,30.0,1,80
13,2023-01-28,13,Patchy rain possible,30.9,1,75
15,2023-01-28,15,Patchy light rain with thunder,31.1,1,72


In [10]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [54]:
df['Fecha'][0]

'2023-01-28'

In [ ]:
df_rain

In [55]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-01-28 en Argentina es : \n\n\n                            Condicion\nHora                                \n12              Patchy rain possible\n13              Patchy rain possible\n15    Patchy light rain with thunder'

In [ ]:
PHONE_NUMBER

# Mensaje SMS desde Twilio

In [37]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages\
                .create(
                    #  body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     body= 'Brunito, sos un pajin' ,
                     from_='whatsapp:+14155238886',
                     to=f'whatsapp:+5491121718650'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMf8bec0e59f7647bd96e6c1b6a9fe32bd


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />